# Wstęp
W pierwszej części projektu skupiami się na scrapowaniu danych z różnych stron internetowych z przepisami kulinarnymi. Celem jest zebranie informacji o przepisach takich jak tytuł, składniki, sposób przygotowania, ocena, liczba opinii, czas przygotowania oraz czy przepis jest wegetariański.

## Import bibliotek

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from time import sleep
import json
import re
from collections import Counter
from wordcloud import WordCloud
import time

colab_start_time = time.time()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/My Drive/PK-DS-JZTW-colabs/our_functions.py' /content/

In [ ]:
import our_functions

In [ ]:
# W celu monitorowania czasu wykonywania kodu, definiujemy funkcje startu i stopu stopera

# %%writefile our_functions.py

def start_time_meter():
  # Uruchomienie stopera
  print("Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.")
  return time.time()

def stop_time_meter(start_time):
  # Zatrzymanie stopera
  end_time = time.time()

  # Obliczenie czasu wykonania
  execution_time = end_time - start_time

  # Wyświetlenie czasu wykonania
  return print(f"Czas wykonania: {execution_time:.2f} sekundy")

In [ ]:
# Ustawiamy nagłówek dla wszystkich pobrań
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

## Scrapowanie pierwszej strony z przepisami (kwestiasmaku.com)


In [ ]:
# Pobieramy url na podstawie kategorii i numeru strony
def get_recipe_page_kwestiasmaku(base_url, category, page_number, include_przepisy_html=True):
    if page_number == 1:
        url = f'{base_url}/{category}' + ('/przepisy.html' if include_przepisy_html else '')
    else:
        url = f'{base_url}/{category}' + ('/przepisy.html' if include_przepisy_html else '') + f'?page={page_number}'
    print(f'Pobieranie URL: {url}')  # Dodano dla debugowania
    content = requests.get(url).text
    # Zwracany jest obiekt BeautifulSoup który zawiera przetworzoną stonę HTML
    return BeautifulSoup(content, 'html.parser')

# Pobieramy szczegółowe informacje o przepisie
def get_recipe_details_kwestiasmaku(recipe_url, category):
    response = requests.get(recipe_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # znajdowanie tytułu
    title_element = soup.find('h1', {'class': 'przepis page-header'})
    recipe_title = title_element.get_text(strip=True) if title_element else 'Brak tytułu'
    # znajdowanie składników
    ingredients_list = soup.find('div', {'class' : 'field field-name-field-skladniki field-type-text-long field-label-hidden'})
    ingredients = [li.get_text(strip=True) for li in ingredients_list.find_all('li')] if ingredients_list else []
    # znajdowanie oceny
    rating_element = soup.find('span', {'itemprop': 'ratingValue'})
    rating = rating_element.get_text(strip=True) if rating_element else 'Brak oceny'
    # znajdowanie liczby opinii
    opinions = soup.find('span', {'itemprop': 'reviewCount'})
    opinions_count = opinions.get_text(strip=True) if opinions else 'Brak opinii'
    # znajdowanie sposobu przygotowania
    preparation_method_container = soup.find('div', {'class' : 'field field-name-field-przygotowanie field-type-text-long field-label-above'})
    preparation_method = preparation_method_container.get_text(strip=True) if preparation_method_container else 'Brak sposobu przygotowania'

    return {'category': category, 'recipe_title': recipe_title, 'ingredients': ';'.join(ingredients), 'preparation_method': preparation_method, 'rating': rating, 'opinions_count': opinions_count}

# Iterujemy przez określoną liczę stron w danej kategorii, pobierając i zbierając szcegóły przepisów
def get_recipes_data_kwestiasmaku(category, number_of_pages, base_url, include_przepisy_html=True):
    all_recipes = []
    for page_number in range(1, number_of_pages + 1):
        soup = get_recipe_page_kwestiasmaku(base_url, category, page_number, include_przepisy_html)
        recipes = soup.find_all('div', {'class': 'col col-lg-3'})
        for recipe in recipes:
            link_element = recipe.find('a', href=True)
            if link_element:
                recipe_link = 'https://www.kwestiasmaku.com' + link_element['href']
                print(recipe_link)
                details = get_recipe_details_kwestiasmaku(recipe_link, category)
                if details:  # Dodajemy tylko, jeśli udało się pobrać szczegóły
                    all_recipes.append(details)
        print(f'Downloaded data from page {page_number}')
    return all_recipes


In [ ]:
!ls

drive  our_functions.py  __pycache__  sample_data


In [ ]:
# Ten fragment kodu pobiera dane przepisów z różnych kategorii oraz mierzy czas wykonania każdego z pobrań
start_time = our_functions.start_time_meter()
breakfast_data = our_functions.get_recipes_data_kwestiasmaku('sniadania', 8, 'https://www.kwestiasmaku.com/dania_dla_dwojga', include_przepisy_html=True)
our_functions.stop_time_meter(start_time)
print('-----')
start_time = start_time_meter()
main_dish_data = our_functions.get_recipes_data_kwestiasmaku('dania-obiadowe', 8, 'https://www.kwestiasmaku.com/blog-kulinarny/category', include_przepisy_html=False)
stop_time_meter(start_time)
print('-----')
start_time = start_time_meter()
dinner_data = our_functions.get_recipes_data_kwestiasmaku('kolacje', 8, 'https://www.kwestiasmaku.com/przepisy', include_przepisy_html=False)
stop_time_meter(start_time)
print('-----')

Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.
Pobieranie URL: https://www.kwestiasmaku.com/dania_dla_dwojga/sniadania/przepisy.html
https://www.kwestiasmaku.com/przepis/szparagi-zapiekane-z-jajkiem
https://www.kwestiasmaku.com/przepis/croque-mademoiselle
https://www.kwestiasmaku.com/przepis/power-bowl-z-batatami-awokado-i-jajkiem
https://www.kwestiasmaku.com/przepis/jajka-z-rzezucha
https://www.kwestiasmaku.com/przepis/pasta-z-fasoli
https://www.kwestiasmaku.com/przepis/jajka-z-majonezem-truflowym
https://www.kwestiasmaku.com/przepis/pan-bagnat
https://www.kwestiasmaku.com/przepis/tosty-francuskie-z-solonym-karmelem
https://www.kwestiasmaku.com/przepis/menemen
https://www.kwestiasmaku.com/przepis/tosty-z-omletem
https://www.kwestiasmaku.com/przepis/pieczony-nalesnik-dutch-baby-z-malinami
https://www.kwestiasmaku.com/przepis/tosty-francuskie-z-truskawkami
https://www.kwestiasmaku.com/przepis/szparagi-z-jajkami-i-tartym-serem-dojrzewajacym
https://www.kwestiasmaku.com/przepis/torti

In [ ]:
df = pd.DataFrame(breakfast_data + main_dish_data + dinner_data)

In [ ]:
df.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count
0,sniadania,Szparagi zapiekane z jajkiem,1/2 pęczka szparagów;1 łyżeczka masła;100 g po...,Piekarnik nagrzać do180 stopni C. Dno naczynia...,4.79,14
1,sniadania,Croque Mademoiselle,4 duże kromki chleba np. na zakwasie;1 cukinia...,"Cukinię umyć, odciąć końce, pokroić na ok. 0,5...",4.75,4
2,sniadania,"Power bowl z batatami, awokado i jajkiem","1 batat;1/2 ząbka czosnku;sól, pieprz, 1/2 łyż...",Piekarnik nagrzać do 210 stopni C. Batata obra...,4.71,7
3,sniadania,Jajka z rzeżuchą,10 jajek;4 łyżki majonezu;2 łyżeczki musztardy...,Jajka ugotować na twardo (6 minut od zagotowan...,4.5,4
4,sniadania,Pasta z fasoli,1 puszka białej fasolki (400 g);150 g korzenia...,Fasolkę odcedzić na sitku. Selera obrać i pokr...,5,3


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   category            763 non-null    object
 1   recipe_title        763 non-null    object
 2   ingredients         763 non-null    object
 3   preparation_method  763 non-null    object
 4   rating              763 non-null    object
 5   opinions_count      763 non-null    object
dtypes: object(6)
memory usage: 35.9+ KB


Dzięki funkcji df.info() dowiadujemy się o jego strukturze, wiemy że jest to DataFrame, że zawiera 763 wiersze z indeksami o 0 do 762, że zawiera 6 kolumn, że nie ma brakujących danch (non-null), ze typy danych dla wszytskich kolumn to object czyli stringi.

In [ ]:
df.to_csv('recipes_kwestiasmaku.csv', index=False)
df.to_parquet('recipes_kwestiasmaku.parquet', index=False)

In [ ]:
df = pd.read_csv('recipes_kwestiasmaku.csv')
df.head()

## Scrapowanie drugiej strony z przepisami (przepisy.pl)

In [ ]:
# Pobieramy url na podstawie kategorii i numeru strony
def get_recipe_page_przepisy(category, page_number):
    url = f'https://www.przepisy.pl/przepisy/posilek/{category}?page={page_number}'
    response = requests.get(url, headers=headers)
    # sleep(2)  # Dodanie opóźnienia, aby nie obciążać serwera
    return BeautifulSoup(response.content, 'html.parser')

# Pobieramy szczegółowe informacje o przepisie
def get_recipe_details_przepisy(recipe_url, category):
    response = requests.get(recipe_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # znajdowanie tytułu
    title_element = soup.find('h1', {'class': 'title'})
    recipe_title = title_element.get_text(strip=True) if title_element else 'Brak tytułu'
    # znajdowanie składników
    ingredients_list = soup.find_all('span', {'class': 'text-bg-white'})
    ingredients = '; '.join([ingredient.get_text(strip=True) for ingredient in ingredients_list])
    # znajdowanie sposobu przygotowania
    preparation_method_element = soup.find('div', {'class': 'steps-list-wrapper ng-star-inserted'})
    preparation_method = preparation_method_element.get_text(strip=True) if preparation_method_element else 'Brak sposobu przygotowania'
    # znajdowanie oceny
    rating_elements_full = soup.find_all('app-asset-ico-rating', class_='ico-rating ng-star-inserted')
    rating_elements_half = soup.find_all('app-asset-ico-rating', class_='ico-rating last ng-star-inserted')
    rating = len(rating_elements_full) + len(rating_elements_half) * 0.5
    # znajdowanie liczby opinii
    opinions_count_element = soup.find('div', {'class': 'comments-list-container ng-star-inserted'})
    opinions_count_p = opinions_count_element.find('p', class_='average-count rating-total') if opinions_count_element else None
    opinions_count = opinions_count_p.get_text(strip=True) if opinions_count_p else 'Brak opinii'
    # znajdowanie czasu przygotowania
    preparation_time_element = soup.find('div', {'class': 'time-count'})
    preparation_time = preparation_time_element.get_text(strip=True) if preparation_time_element else 'Brak informacji'
    # sprawdzanie czy danie jest wegetariańskie
    vegetarian_element = soup.find('a', href="/przepisy/wege")
    vegetarian = bool(vegetarian_element and 'Wege' in vegetarian_element.text)

    return {'category': category, 'recipe_title': recipe_title, 'ingredients': ingredients, 'preparation_method': preparation_method, 'rating': rating, 'opinions_count': opinions_count, 'preparation_time': preparation_time, 'vegetarian': vegetarian, }

# Iterujemy przez określoną liczę stron w danej kategorii, pobierając i zbierając szcegóły przepisów
def get_recipes_data_przepisy(categories, number_of_pages):
    all_recipes = []
    for category in categories:
        for page_number in range(1, number_of_pages + 1):
            soup = get_recipe_page_przepisy(category, page_number)
            if not soup:
              continue
            links = soup.find_all('a', class_='recipe-box__title')
            for link in links:
                recipe_url = 'https://www.przepisy.pl' + link['href']
                recipe_data = get_recipe_details_przepisy(recipe_url, category)
                if recipe_data:
                  all_recipes.append(recipe_data)
    return all_recipes

In [ ]:
start_time = start_time_meter()
breakfast_data_1 = get_recipes_data_przepisy(['sniadanie'], 9)
print(f'Liczba przepisów: {len(breakfast_data_1)}')
print('-----')
stop_time_meter(start_time)

Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.
Liczba przepisów: 272
-----
Czas wykonania: 243.80 sekundy


In [ ]:
start_time = our_functions.start_time_meter()
main_dish_data_1 = get_recipes_data_przepisy(['obiad'], 20)
print(f'Liczba przepisów: {len(main_dish_data_1)}')
print('-----')
our_functions.stop_time_meter(start_time)

Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.
Liczba przepisów: 620
-----
Czas wykonania: 715.23 sekundy


In [ ]:
start_time = our_functions.start_time_meter()
dinner_data_1 = get_recipes_data_przepisy(['kolacja'], 20)
print(f'Liczba przepisów: {len(dinner_data_1)}')
print('-----')
our_functions.stop_time_meter(start_time)

Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.
Liczba przepisów: 620
-----
Czas wykonania: 618.72 sekundy


In [ ]:
df_1 = pd.DataFrame(breakfast_data_1 + main_dish_data_1 + dinner_data_1)

In [ ]:
df_1.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian
0,sniadanie,Szakszuka,jajka; 4 sztuki; Przyprawa do warzyw z grilla ...,Krok 1Sparz pomidory do szakszuki z piekarnika...,4.5,Brak opinii,25 min.,True
1,sniadanie,Hiszpański omlet,jajko; 5 sztuk; Majonez Hellmann's Oryginalny;...,Krok 1Przygotuj wszystkie składniki: cebulę i ...,4.5,Brak opinii,30 min.,True
2,sniadanie,Kremowa kanapka z indykiem,"chleb żytni na zakwasie, kromki; 4 sztuki; pie...","Krok 1Rozgrzej patelnię grillową lub opiekacz,...",4.5,Brak opinii,15 min.,True
3,sniadanie,Kanapki z pastą rybną z jajkiem i ogórkiem kon...,Majonez Hellmann's Oryginalny; 3 łyżki; makrel...,Krok 1Jaja ugotuj na twardo obierz zetrzyj na ...,4.5,Brak opinii,15 min.,True
4,sniadanie,"Kanapka BLT (bekon, pomidor, sałata)",kromki chleba tostowego; 8 sztuk; Majonez Hell...,Krok 1Na patelni na średnim ogniu usmażyć bocz...,4.5,Brak opinii,10 min.,True


In [ ]:
df_1.to_csv('recipes_przepisy.csv', index=False)
df_1.to_parquet('recipes_przepisy.parquet', index=False)

In [ ]:
df_1 = pd.read_csv('recipes_przepisy.csv')

In [ ]:
df_1.head()

## Scrapowanie trzeciej strony z przepisami (aniastarmach.pl) TW

In [ ]:
# Pobieramy strony z HTML z danego URL i zwracanie jej zawartości jako obiektu BeautifulSoup
def get_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 404:  # Sprawdzenie, czy strona nie istnieje
        return None
    return BeautifulSoup(response.text, 'html.parser')

# Pobieramy nazw i URL-ów przepisów z danej strony kategorii
def get_recipes_names_urls_for_page(page_url):
    soup = get_page(page_url)
    if soup is None:  # Jeśli strona nie istnieje, zwróć puste listy
        return [], [], True
    recipes_names, recipes_urls = [], []
    recipes = soup.find_all('div', class_='recipe')
    for recipe in recipes:
        title = recipe.get('data-title')
        recipe_url = recipe.find('a', class_='overlay')['href']
        recipes_names.append(title)
        recipes_urls.append(recipe_url)
    return recipes_names, recipes_urls, False

# Pobieranie szczegółowych informacji o przepisach
def get_receipes_details(recipes_urls, category):
    recipes_data = []
    for url in recipes_urls:
        soup = get_page(url)
        if soup is None:
            continue

        # Pobieranie i czyszczenie nazwy potrawy
        recipe_title = soup.find('div', class_='recipe').get('data-title', 'Brak tytułu')
        recipe_title = recipe_title.replace('\n', '').replace(';', '=')

        # Pobieranie i czyszczenie składników
        items = soup.find('div', class_='recipe-what-to-buy')
        ingredients_descriptions = []
        if items:
            items = items.find_all('li')
        for item in items:
            quantity = item.find('span', class_='quantity')
            quantity = quantity.text.strip() if quantity else 'do smaku'
            ingredient_name = item.text.strip().replace(quantity, '', 1).strip()
            description = f"{quantity} {ingredient_name}" if quantity != 'do smaku' else ingredient_name
            ingredients_descriptions.append(description)
        ingredients = '; '.join(ingredients_descriptions)
        ingredients = ingredients.replace('\n', '').replace(';', '=')

        # Pobieranie i czyszczenie sposobu przygotowania
        steps_section = soup.find('div', class_='recipe-steps')
        preparation = "Brak informacji"
        if steps_section:
            steps = steps_section.find_all('li')
            preparation_steps = []
            for step in steps:
                p = step.find('p')
                step_text = p.text.strip() if p else step.text.strip()
                preparation_steps.append(step_text)
            preparation = "\n\n".join(preparation_steps)
        preparation = preparation.replace('\n', '').replace(';', '=')

        # Pozostałe elementy
        time_section = soup.find('div', class_='recipe-icon icon-timer')
        preparation_time = time_section.span.text.strip() + " min" if time_section and time_section.span else "Brak informacji"

        vegetarian_icon = soup.find('div', class_='recipe-icon icon-vegetarian')
        is_vegetarian = bool(vegetarian_icon)

        recipes_data.append({
            'category': category,
            'recipe_title': recipe_title,
            'ingredients': ingredients,
            'preparation_method': preparation,
            'preparation_time': preparation_time,
            'vegetarian': is_vegetarian,
        })

    return recipes_data


In [ ]:
start_time = start_time_meter()

categories = ['sniadanie', 'obiad', 'kolacja']

all_recipes_details = []

for category in categories:
    recipes_names, recipes_urls = [], []
    base_url = f"https://aniastarmach.pl/przepisy/pora-dnia/{category}/strona/"
    page_number = 1
    while (page_number < 10):
        names, urls, not_found = get_recipes_names_urls_for_page(base_url + str(page_number) + "/")
        if not_found:
            break
        recipes_names.extend(names)
        recipes_urls.extend(urls)
        page_number += 1

    category_recipes_details = get_receipes_details(recipes_urls, category)
    all_recipes_details.extend(category_recipes_details)

print('-----')
stop_time_meter(start_time)

Rozpoczęcie pomiaru czasu wykonania fragmentu kodu.
-----
Czas wykonania: 580.53 sekundy


In [ ]:
df_2 = pd.DataFrame(all_recipes_details)

In [ ]:
df_2.head()

,category,recipe_title,ingredients,preparation_method,preparation_time,vegetarian
0,sniadanie,Minifrittaty,4 jajka= 1 awokado= 1 łyżeczka soku z cytryny=...,"Jajka ugotuj na twardo, obierz ze skorupek, pr...",15 min,True
1,sniadanie,Placuszki z kuskusu i białego sera,4 łyżki kaszy jaglanej= 1 szklanka mleka kokos...,"Kaszę przelej zimną, potem ciepłą wodą. Przełó...",20 min,True
2,sniadanie,Minifrittaty,4 duże kromki chleba= 100 g sera ricotta= 1 ły...,"Szynkę pokrój w kostkę, wrzuć na rozgrzaną pat...",10 min,True
3,sniadanie,Tosty trójkąciki,5 łyżek płatków owsianych górskich= 200 ml mle...,Płatki owsiane przełóż do niewielkiego rondelk...,20 min,True
4,sniadanie,Naleśniki „ruskie”,4 jajka= 50 ml mleka= 0.5 łyżki masła= 8 pomid...,"Szynkę pokrój w cienkie paseczki, mozzarellę –...",15 min,False


In [ ]:
# Dodajemy dwie nowe kolumny do naszego df ponieważ w innych stronach te kolumny występowały
prep_method_idx = df_2.columns.get_loc('preparation_method') + 1
df_2.insert(prep_method_idx, 'rating', np.nan)
df_2.insert(prep_method_idx + 1, 'opinions_count', np.nan)

In [ ]:
df_2.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian
0,sniadanie,Minifrittaty,4 jajka= 1 awokado= 1 łyżeczka soku z cytryny=...,"Jajka ugotuj na twardo, obierz ze skorupek, pr...",NaN,NaN,15 min,True
1,sniadanie,Placuszki z kuskusu i białego sera,4 łyżki kaszy jaglanej= 1 szklanka mleka kokos...,"Kaszę przelej zimną, potem ciepłą wodą. Przełó...",NaN,NaN,20 min,True
2,sniadanie,Minifrittaty,4 duże kromki chleba= 100 g sera ricotta= 1 ły...,"Szynkę pokrój w kostkę, wrzuć na rozgrzaną pat...",NaN,NaN,10 min,True
3,sniadanie,Tosty trójkąciki,5 łyżek płatków owsianych górskich= 200 ml mle...,Płatki owsiane przełóż do niewielkiego rondelk...,NaN,NaN,20 min,True
4,sniadanie,Naleśniki „ruskie”,4 jajka= 50 ml mleka= 0.5 łyżki masła= 8 pomid...,"Szynkę pokrój w cienkie paseczki, mozzarellę –...",NaN,NaN,15 min,False


In [ ]:
df_2.to_csv('recipes_aniastarmach.csv',index=False)
df_2.to_parquet('recipes_aniastarmach.parquet',index=False)

In [ ]:
df_2 = pd.read_csv('recipes_aniastarmach.csv')

In [ ]:
df_2.head()

## Składanie w jednego df_recipes

In [ ]:
# Łączymy 3 obiekty DataFrame, argument ignore_index ignoruje indexy i ponownie zaindeksuje od 0
df_recipes = pd.concat([df, df_1, df_2], ignore_index=True)

In [ ]:
df_recipes.head()

,category,recipe_title,ingredients,preparation_method,rating,opinions_count,preparation_time,vegetarian
0,sniadania,Szparagi zapiekane z jajkiem,1/2 pęczka szparagów;1 łyżeczka masła;100 g po...,Piekarnik nagrzać do180 stopni C. Dno naczynia...,4.79,14,NaN,NaN
1,sniadania,Croque Mademoiselle,4 duże kromki chleba np. na zakwasie;1 cukinia...,"Cukinię umyć, odciąć końce, pokroić na ok. 0,5...",4.75,4,NaN,NaN
2,sniadania,"Power bowl z batatami, awokado i jajkiem","1 batat;1/2 ząbka czosnku;sól, pieprz, 1/2 łyż...",Piekarnik nagrzać do 210 stopni C. Batata obra...,4.71,7,NaN,NaN
3,sniadania,Jajka z rzeżuchą,10 jajek;4 łyżki majonezu;2 łyżeczki musztardy...,Jajka ugotować na twardo (6 minut od zagotowan...,4.5,4,NaN,NaN
4,sniadania,Pasta z fasoli,1 puszka białej fasolki (400 g);150 g korzenia...,Fasolkę odcedzić na sitku. Selera obrać i pokr...,5,3,NaN,NaN


In [ ]:
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2563 entries, 0 to 2562
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   category            2563 non-null   object
 1   recipe_title        2563 non-null   object
 2   ingredients         2563 non-null   object
 3   preparation_method  2563 non-null   object
 4   rating              2275 non-null   object
 5   opinions_count      2275 non-null   object
 6   preparation_time    1800 non-null   object
 7   vegetarian          1800 non-null   object
dtypes: object(8)
memory usage: 160.3+ KB


In [ ]:
df_recipes.to_csv('df_recipes.csv', index=False)

In [ ]:
df_recipes = pd.read_csv('df_recipes.csv')

In [ ]:
!cp * '/content/drive/My Drive/'

cp: -r not specified; omitting directory 'drive'
cp: -r not specified; omitting directory '__pycache__'
cp: -r not specified; omitting directory 'sample_data'


# Wnioski
Udało się skutecznie pobrać dane z trzech różnych źródeł stron internetowych z przepisami kulinarnymi oraz zapisaniem ich do jednego DataFrame zawierające informację takie jak tytuł, składniki, metoda przygotowania, ocena oraz liczba opinii, dzięki czemu możemy przejść do kolejnego etapu jakim jest analiza danych.